In [1]:
import s1_prepare_data as s1
import pandas as pd

'input current date: '

20191008


In [2]:
PD = s1.Prepare_Data()

In [3]:
PD.login_and_choose_database(database = 'python_clickhouse_test')

In [4]:
PD.raw_query('show tables;')

[('account_table',),
 ('basic_table',),
 ('basic_table_sample',),
 ('fraud_case_table',),
 ('information_table',),
 ('merged_table_1',)]

In [5]:
PD.raw_query('desc basic_table;')

[('cust_id', 'String', '', '', '', ''), ('date', 'Date', '', '', '', '')]

In [6]:
PD.raw_query('desc information_table;')

[('cust_id', 'String', '', '', '', ''),
 ('date', 'Date', '', '', '', ''),
 ('gender', 'String', '', '', '', ''),
 ('job', 'String', '', '', '', '')]

## merge data

In [4]:
PD.merge_data(new_table = 'merged_table_1',
             ordered_column = 'cust_id',
             joined_method = 'inner join',
             left_table_query = ' select cust_id as cust_id_basic, date as date_basic from basic_table',
             right_table_query = ' select * from information_table',
             on_query = 'a.cust_id_basic = b.cust_id \n AND a.date_basic = b.date'
             )

query overview: 
CREATE TABLE IF NOT EXISTS merged_table_1
ENGINE = MergeTree
ORDER BY cust_id
AS SELECT * FROM ( 
 select cust_id as cust_id_basic, date as date_basic from basic_table)a 
inner join( 
 select * from information_table)b 
ON a.cust_id_basic = b.cust_id 
 AND a.date_basic = b.date

show tables: 
[('account_table',), ('basic_table',), ('basic_table_sample',), ('fraud_case_table',), ('information_table',), ('merged_table_1',)]


In [5]:
PD.get_dataframe_head(table_name = 'merged_table_1')

,cust_id_basic,date_basic,cust_id,date,gender,job
0,2000007878,2018-12-19,2000007878,2018-12-19,Male,~
1,2000008555,2018-11-01,2000008555,2018-11-01,Female,Engineer
2,2000024405,2018-05-28,2000024405,2018-05-28,Male,Teacher
3,2000031035,2018-08-10,2000031035,2018-08-10,~,Students
4,2000031841,2018-11-05,2000031841,2018-11-05,Female,Accounter


In [6]:
PD.raw_query('select count() from merged_table_1')

[(500000,)]

In [ ]:
!clickhouse-client --host="localhost" --port="9000" --user="default" --max_threads="1"  --format_csv_delimiter="|" \
--query="select * from python_clickhouse_test.merged_table_1 format CSV"  > merged_table_1
